## Implement DH with negotiated groups, and break with malicious "g" parameters

```
A->B
    Send "p", "g"
B->A
    Send ACK
A->B
    Send "A"
B->A
    Send "B"
A->B
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv
B->A
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), A's msg) + iv 
```

 Do the MITM attack again, but play with "g". What happens with:

```
    g = 1
    g = p
    g = p - 1
```

Write attacks for each. 

Alice randomly selects private value $a$ and calculates public value $A = g^{a}\mod{p}$, which is sent to Bob.

For $g = 1$, Alice sends $A = 1^a\mod{p} = 1$. Bob calculates shared secret $s = A^b\mod{p} = 1^b\mod{p} = 1$. In other words, if MITM attacker can inject a malicious $g = 1$ parameter into the DH group negotiation, the shared secret is forced to have value $1$.

For $g = p$, Alice sends $A = p^a\mod{p}$. Bob calculates shared secret $s = A^b\mod{p} = (p^a)^b\mod{p} = p^{ab}\mod{p} = 0$, so the shared secret is forced to the value of $0$.

Not sure what happens for $g = p - 1$, let's play.

In [1]:
p = int(
"ffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024"
"e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd"
"3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec"
"6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f"
"24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361"
"c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552"
"bb9ed529077096966d670c354e4abc9804f1746c08ca237327fff"
"fffffffffffff", 16)
p

2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919

In [2]:
g = p - 1

In [3]:
from random import randrange

In [4]:
a = randrange(1, p)
a

853985539902227884792671979705632874992389566317062556262742515110945573893579475326834860981702986870595144037196892933915151564196802799565929399956246202707874244073957760337675350870226504768944862154251766648823075762066757117085521672932617139697744479465365108828317550371946175359975103444430190282122477120328089560344462040540545020808677188460569116705213355417016981351770108668295764804228932383447297493663713504621647155461541938556200424670119178

In [5]:
A = pow(g, a, p)
A

1

For $g = p - 1$, Alice sends $A = (p - 1)^a\mod{p}$ to Bob.

If $a = 1$, then $A = p - 1$. 

If $a = 2$, then $A = (p - 1)^2\mod{p} = p^2 - 2p + 1\mod{p} = 1$

If $a$ is odd, then $A = p - 1$. If $a$ is even, then $A = 1$.

### When does this ever happen?
Honestly, not that often in real-world systems. If you can mess with "g", chances are you can mess with something worse. Most systems pre-agree on a static DH group. But the same construction exists in Elliptic Curve Diffie-Hellman, and this becomes more relevant there.
